In [1]:
import json
import numpy as np
import networkx as nx
import tqdm
import os
import yaml
import pickle

path_to_workload = "/home/lenovo/Downloads/trace_0_FLOW_THROUGHPUT-1250000__SIMULATION_TIME-100000000__PAIRS_PER_SRC-1-0__MSG_SIZE-10000__PACKET_SIZE-1400__BANDWIDTH-1250000__PRIO_LEVELS-3"
path_to_topology = "/home/lenovo/Documents/GitHub/experiment-utils/src/data/reindexed.json"
subgraph_number = 0


# read topo data
with open(path_to_topology,'r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']
NUM_NODES, NUM_EDGES = len(nodes), len(edges)

# creating some usefull maps to get information regarding the nodes
try:
    new_to_old = {}
    for node in nodes:
        new_to_old[node['id']] = node['old_id']
except KeyError:
    print("OLD_ID in topology... ingoring new_to_old structure")

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port


In [2]:
g = nx.DiGraph()

connection_info = {node['id']: {} for node in nodes}

for edge in edges:
    src, src_port, dest, dest_port = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']
    src_bw, dest_bw = port_map[src][src_port]['bw'], port_map[dest][dest_port]['bw']
    
    connection_info[src][dest] = port_map[src][src_port]

    g.add_edge(src, dest)

print(g)

DiGraph with 8979 nodes and 21082 edges


In [3]:
# get the nodes of the subgraphs in the topology
subgraphs = list(nx.strongly_connected_components(g))

# get the topology of selected subgraph
g = g.subgraph(subgraphs[subgraph_number])

print(g)

DiGraph with 5237 nodes and 12149 edges


In [4]:
flows = {}
with open(path_to_workload, 'r') as f:
    for line in f:
        src, dest = int(line.split(' ')[1]), int(line.split(" ")[2])

        if src in flows.keys():
            flows[src].add(dest)
        else:
            flows[src] = set([dest])

In [57]:
partitions = {}

''' 
    IF PARTITIONS ARE IN A FILE PARSE IT HERE
    IF THE PARTITIONING IS SIMPLE, IMPLEMENT THE LOGIC IN THE FOR LOOP
'''

num_partitions = 16

for node in nodes:
    partitions[node['id']] = node['id'] % num_partitions


In [58]:
def analyse_flow(route, partitions):
    partitions_span = set()
    
    for node_id in route:
        partitions_span.add(partitions[node_id])
    
    return len(partitions_span)


In [59]:
results = []
for src, flow_set in flows.items():
    flow_list = list(flow_set)

    for dst in flow_list:
        path = nx.shortest_path(g, src, dst)
        span = analyse_flow(path, partitions)
        
        results.append({
            "src": src,
            "dest": dst,
            "span": span
        })
    

In [60]:
spans = [x['span'] for x in results]

count_spans = {}
for span in spans:
    count_spans[span] = count_spans.get(span, 0) + 1

count_spans = [(key, value) for key, value in count_spans.items()]
count_spans = sorted(count_spans, key=lambda x:x[0])
count_spans = {x[0]:x[1] for x in count_spans}

print(num_partitions, count_spans)

16 {1: 2, 2: 5, 3: 7, 4: 49, 5: 149, 6: 307, 7: 611, 8: 788, 9: 971, 10: 933, 11: 681, 12: 371, 13: 193, 14: 59, 15: 21, 16: 2}


number_partitions   {span: num_flows_with_that_span_in_workload, ...}
2   {2: 5132, 1: 17}
4   {1: 3, 2: 37, 3: 715, 4: 4394}
8   {1: 2, 2: 9, 3: 36, 4: 200, 5: 705, 6: 1487, 7: 1723, 8: 987}
16  {1: 2, 2: 5, 3: 7, 4: 49, 5: 149, 6: 307, 7: 611, 8: 788, 9: 971, 10: 933, 11: 681, 12: 371, 13: 193, 14: 59, 15: 21, 16: 2}

span: How many partitions a flow spans accross. A flow in the same partition spans 1 partition. if the switches in the flow are part of three partitions then the span is 3 etc...7 
Example: for the example with 2 partitions, 5132 flows span accross 2 partions and only 17 are within a partition